# Crime Analyzer — General — Data Cleaning (Preprocessing)

Purpose
- Clean and standardize NYPD complaints prior to spatial enrichment and feature engineering
- Remove corrupt/irrelevant records while preserving distributions of key variables

Inputs
- JupyterOutputs/PrePreProcessed/cleaned_crime_data.csv

Outputs
- JupyterOutputs/Processed/cleaned_crime_data_processed.csv

Design decisions (non-obvious)
- Coordinates: remove rows with missing lat/lon; if >5% missing, remove stratified by BORO_NM/LAW_CAT_CD/OFNS_DESC to preserve class mix
- PD_CD: drop if missing because downstream mapping requires it
- Placeholders: normalize common placeholders like (NULL)/UNKNOWN with uppercase casing for consistency
- Age groups: enforce patterns (e.g., 25-44, <18, 65+, UNKNOWN); drop invalid formats
- Categorical text: uppercase and trim; map borough abbreviations and location variants (e.g., FRONT OF → FRONT)
- Duplicates: drop exact duplicates only

Reproducibility
- All artifacts are written under JupyterOutputs; no in-place mutations of source files

## Imports

Use pandas/numpy for tabular ops, seaborn/matplotlib for quick checks, and sklearn utilities for optional strategies. Warnings are suppressed for signal-to-noise.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from datetime import datetime
from collections import Counter
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

## Helper functions

Utilities for missingness profiling, placeholder/string anomaly detection, and validation planning that guide cleaning actions without hard-failing the pipeline.

In [ ]:
# Helper functions for efficient data analysis
def analyze_missing_values(df):
    """Comprehensive missing value analysis"""
    result = {}

    # Standard null/NaN analysis
    null_counts = df.isna().sum()
    null_percentages = (null_counts / len(df)) * 100
    result['standard_nulls'] = pd.DataFrame({
        'Column': null_counts.index,
        'Null_Count': null_counts.values,
        'Null_Percentage': null_percentages.values
    })[null_counts > 0].sort_values('Null_Count', ascending=False)

    return result

def detect_placeholder_values(df, placeholder_list=None):
    """Detect common placeholder values in object columns"""
    if placeholder_list is None:
        placeholder_list = [
            'NA', 'N/A', 'NONE', 'UNKNOWN', 'NULL', '-', '--', '?', '0', 'MISSING', 'U',
            'UNDEFINED', 'NOT AVAILABLE', 'NOT APPLICABLE', 'NOT SPECIFIED', 'UNSPECIFIED',
            'NOT DETERMINED', 'NOT REPORTED', 'NR', 'NIL', 'BLANK', 'UNK', '.', '..', '...',
            '9999', '999', 'TBD', 'PENDING', '(NULL)'
        ]

    placeholder_values_upper = [p.upper() for p in placeholder_list]
    placeholder_counts = {}

    for col in df.select_dtypes(include=['object']).columns:
        if df[col].notna().any():
            try:
                col_upper = df[col].astype(str).str.upper()
                found_placeholders = col_upper.isin(placeholder_values_upper)
                if found_placeholders.any():
                    counts = col_upper[found_placeholders].value_counts()
                    placeholder_counts[col] = counts[counts.index.isin(placeholder_values_upper)]
            except Exception as e:
                print(f"Warning: Could not process column {col}: {e}")

    return placeholder_counts

def detect_string_anomalies(df):
    """Detect empty strings, whitespace-only, and special character strings"""
    empty_counts = {}
    whitespace_counts = {}
    special_char_counts = {}

    for col in df.select_dtypes(include=['object']).columns:
        try:
            df_col_str = df[col].astype(str)

            # Empty strings
            empty_count = (df_col_str == '').sum()
            if empty_count > 0:
                empty_counts[col] = empty_count

            # Whitespace only
            whitespace_count = df_col_str.str.isspace().sum()
            if whitespace_count > 0:
                whitespace_counts[col] = whitespace_count

            # Special characters only (excluding specific placeholders)
            special_chars_only = df_col_str.str.match(r'^[^a-zA-Z0-9\s]+$').sum()
            if special_chars_only > 0:
                unique_special_vals = df_col_str[df_col_str.str.match(r'^[^a-zA-Z0-9\s]+$')].unique()
                # Don't count if only '(NULL)' placeholder
                if not (len(unique_special_vals) == 1 and unique_special_vals[0].upper() == '(NULL)'):
                    special_char_counts[col] = special_chars_only

        except Exception as e:
            print(f"Warning: Could not process column {col} as string: {e}")

    return {
        'empty_strings': empty_counts,
        'whitespace_only': whitespace_counts,
        'special_chars_only': special_char_counts
    }

## Advanced strategies and validation

Heuristics for missing-value strategies, quality gates, and stratified removal to minimize distribution drift.

In [ ]:
# Advanced missing value handling strategy
def missing_value_strategy(df, target_column=None):
    """
    Missing value handling with multiple strategies
    based on data characteristics and ML best practices
    """
    strategies = {}

    for col in df.columns:
        missing_pct = (df[col].isna().sum() / len(df)) * 100
        col_type = df[col].dtype

        if missing_pct == 0:
            strategies[col] = 'no_action'
        elif missing_pct > 90:
            strategies[col] = 'drop_column'  # Too sparse to be useful
        elif missing_pct > 70:
            # High missingness - check if pattern is informative
            if target_column and target_column in df.columns:
                # Test if missingness correlates with target
                missing_mask = df[col].isna()
                if len(df[target_column].unique()) == 2:  # Binary target
                    missing_correlation = abs(df.loc[missing_mask, target_column].mean() -
                                           df.loc[~missing_mask, target_column].mean())
                    if missing_correlation > 0.05:  # Significant difference
                        strategies[col] = 'create_missing_indicator'
                    else:
                        strategies[col] = 'drop_column'
                else:
                    strategies[col] = 'drop_column'
            else:
                strategies[col] = 'drop_column'
        elif missing_pct > 30:
            # Moderate missingness - use advanced imputation
            if pd.api.types.is_numeric_dtype(col_type):
                strategies[col] = 'iterative_imputation'
            else:
                strategies[col] = 'mode_with_indicator'
        elif missing_pct > 5:
            # Low missingness - standard imputation
            if pd.api.types.is_numeric_dtype(col_type):
                strategies[col] = 'median_imputation'
            else:
                strategies[col] = 'mode_imputation'
        else:
            # Very low missingness - simple forward fill or mode
            strategies[col] = 'simple_imputation'

    return strategies

# Data validation framework
def validate_data_quality(df, validation_rules=None):
    """
    Comprehensive data validation framework
    """
    validation_results = {
        'passed': [],
        'warnings': [],
        'errors': []
    }

    # Default validation rules
    if validation_rules is None:
        validation_rules = {
            'min_rows': 1000,
            'max_missing_pct': 50,
            'required_columns': [],
            'numeric_ranges': {},
            'categorical_validity': {}
        }

    # Check minimum rows
    if len(df) < validation_rules['min_rows']:
        validation_results['warnings'].append(
            f"Dataset has only {len(df)} rows, less than recommended minimum {validation_rules['min_rows']}"
        )

    # Check overall missing data percentage
    total_missing_pct = (df.isna().sum().sum() / (len(df) * len(df.columns))) * 100
    if total_missing_pct > validation_rules['max_missing_pct']:
        validation_results['warnings'].append(
            f"Overall missing data percentage ({total_missing_pct:.1f}%) exceeds threshold ({validation_rules['max_missing_pct']}%)"
        )

    # Check for required columns
    for col in validation_rules['required_columns']:
        if col not in df.columns:
            validation_results['errors'].append(f"Required column '{col}' is missing")

    # Validate numeric ranges
    for col, (min_val, max_val) in validation_rules['numeric_ranges'].items():
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            if df[col].min() < min_val or df[col].max() > max_val:
                validation_results['warnings'].append(
                    f"Column '{col}' has values outside expected range [{min_val}, {max_val}]"
                )

    # Check data consistency
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        validation_results['warnings'].append(f"Found {duplicates} duplicate rows")

    return validation_results

# Stratified sampling for data removal
def stratified_data_removal(df, target_column, removal_mask, max_removal_pct=0.1):
    """
    Remove data while maintaining target distribution
    """
    if target_column not in df.columns:
        # If no target column, use simple random sampling
        removal_indices = df[removal_mask].sample(n=min(int(len(df) * max_removal_pct), removal_mask.sum())).index
        return df.drop(removal_indices)

    # Stratified removal to maintain target distribution
    target_distribution = df[target_column].value_counts(normalize=True)
    removal_indices = []

    for target_value in target_distribution.index:
        target_mask = df[target_column] == target_value
        combined_mask = removal_mask & target_mask

        if combined_mask.sum() > 0:
            # Remove proportionally from each target class
            n_to_remove = min(
                int(len(df[target_mask]) * max_removal_pct),
                combined_mask.sum()
            )
            if n_to_remove > 0:
                removal_indices.extend(
                    df[combined_mask].sample(n=n_to_remove).index.tolist()
                )

    return df.drop(removal_indices)

def analyze_missing_values(df):
    """Comprehensive missing value analysis"""
    result = {}

    # Standard null/NaN analysis
    null_counts = df.isna().sum()
    null_percentages = (null_counts / len(df)) * 100

    # Create the DataFrame first
    standard_nulls_df = pd.DataFrame({
        'Column': null_counts.index,
        'Null_Count': null_counts.values,
        'Null_Percentage': null_percentages.values
    })

    # Filter the DataFrame based on the 'Null_Count' column and then sort
    result['standard_nulls'] = standard_nulls_df[standard_nulls_df['Null_Count'] > 0].sort_values('Null_Count', ascending=False)
    missing_patterns = df.isna().value_counts()
    # Handle cases where all values are True or False (all NaN or no NaN) which results in a single item Series
    if len(missing_patterns) > 1:
         result['missing_patterns'] = missing_patterns.head(10)
    else:
         result['missing_patterns'] = pd.Series(dtype='int64') # Return empty Series if no patterns beyond all True/False

    # Correlation between missingness
    missing_df = df.isna().astype(int)
    missing_corr = missing_df.corr()
    # Find high correlations (>0.5) between missing indicators
    high_corr_pairs = []
    for i in range(len(missing_corr.columns)):
        for j in range(i+1, len(missing_corr.columns)):
            # Ensure columns exist in missing_corr
            if missing_corr.columns[i] in missing_corr.index and missing_corr.columns[j] in missing_corr.columns:
                 if abs(missing_corr.iloc[i, j]) > 0.5 and abs(missing_corr.iloc[i, j]) < 1.0: # Exclude self-correlation (always 1.0)
                    high_corr_pairs.append(
                        (missing_corr.columns[i], missing_corr.columns[j], missing_corr.iloc[i, j])
                    )
    result['missing_correlations'] = high_corr_pairs


    return result

## Paths

Read from JupyterOutputs/PrePreProcessed and write to JupyterOutputs/Processed to keep artifacts versionable and easy to collect in CI.

In [ ]:
# Path variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "JupyterOutputs"))
prepreprocessed_dir = os.path.join(base_dir, "PrePreProcessed")
cleaned_data_file = os.path.join(prepreprocessed_dir, "cleaned_crime_data.csv")
processed_dir = os.path.join(base_dir, "Processed")
os.makedirs(processed_dir, exist_ok=True)

# Load data

Load the output from Pre-Preprocessing and print a brief schema/shape summary.

In [5]:
# Load dataset
print("=== Loading PrePreProcessed Data ===")
try:
    if os.path.exists(cleaned_data_file):
        df = pd.read_csv(cleaned_data_file)
        initial_rows = len(df)
        print(f"Dataset loaded successfully: {initial_rows} rows and {df.shape[1]} columns")
        print(f"Columns in the dataset: {df.columns.tolist()}")
    else:
        raise FileNotFoundError(f"Could not find cleaned dataset at: {cleaned_data_file}")
except Exception as e:
    print(f"Error loading dataset: {e}")

=== Loading PrePreProcessed Data ===
Dataset loaded successfully: 2512541 rows and 18 columns
Columns in the dataset: ['BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'Latitude', 'Longitude', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']


## Initial Data Overview

Display basic information and summary statistics of the loaded dataset.

In [6]:
# Display basic dataset overview
print("\n=== Dataset Overview ===")
print(df.info())
print("\n=== Summary Statistics ===")
print(df.describe())


=== Dataset Overview ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512541 entries, 0 to 2512540
Data columns (total 18 columns):
 #   Column             Dtype  
---  ------             -----  
 0   BORO_NM            object 
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   KY_CD              int64  
 4   LAW_CAT_CD         object 
 5   LOC_OF_OCCUR_DESC  object 
 6   Latitude           float64
 7   Longitude          float64
 8   OFNS_DESC          object 
 9   PARKS_NM           object 
 10  PD_CD              float64
 11  PREM_TYP_DESC      object 
 12  SUSP_AGE_GROUP     object 
 13  SUSP_RACE          object 
 14  SUSP_SEX           object 
 15  VIC_AGE_GROUP      object 
 16  VIC_RACE           object 
 17  VIC_SEX            object 
dtypes: float64(3), int64(1), object(14)
memory usage: 345.0+ MB
None

=== Summary Statistics ===
              KY_CD      Latitude     Longitude         PD_CD
count  2.512541e+06  2.512513e+06  2.512513e+06  2.510451e+06

# Analysis

We quantify missingness, placeholders, and anomalies to drive targeted, low-risk cleaning steps. Findings below inform the rules applied in Section 2.x.

## 1.1 Check for Missing and Anomalous Values

Identify standard null/NaN values, empty strings, whitespace-only strings, and common placeholder strings.

### 1.1.1 Standard Null/NaN Analysis

Calculate and display the count and percentage of standard null (NaN) values for each column.

In [ ]:
print("\n=== ANALYSIS SECTION ===")
print("Starting comprehensive analysis of the dataset...")

# 1.1 Advanced Missing Value Analysis
print("\n1.1 COMPREHENSIVE MISSING VALUE ANALYSIS")
print("Performing missing value analysis...")

# Use the enhanced missing value analysis
missing_analysis = analyze_missing_values(df)
null_analysis = missing_analysis['standard_nulls']

print("\nColumns with null/NaN values:")
if not null_analysis.empty:
    print(null_analysis.to_string(index=False))

    # Add missing value strategy recommendations
    print("\n--- Missing Value Strategy Recommendations ---")
    strategies = missing_value_strategy(df)
    for col in null_analysis['Column']:
        strategy = strategies.get(col, 'unknown')
        missing_pct = null_analysis[null_analysis['Column'] == col]['Null_Percentage'].iloc[0]
        print(f"  {col} ({missing_pct:.2f}% missing): {strategy}")
else:
    print("No columns with standard null/NaN values found.")

# Display missing value patterns
if 'missing_patterns' in missing_analysis and len(missing_analysis['missing_patterns']) > 1:
    print("\n--- Top Missing Value Patterns ---")
    for pattern, count in missing_analysis['missing_patterns'].head(5).items():
        pattern_str = ', '.join([f"{col}={val}" for col, val in zip(df.columns, pattern)])
        print(f"  Pattern: {pattern_str} -> {count} rows ({count/len(df)*100:.2f}%)")

# Display correlated missingness
if missing_analysis['missing_correlations']:
    print("\n--- Highly Correlated Missing Values (>0.5) ---")
    for col1, col2, corr in missing_analysis['missing_correlations']:
        print(f"  {col1} <-> {col2}: {corr:.3f}")
else:
    print("\n--- No highly correlated missing values found ---")

print(f"\nTotal columns without null values: {df.shape[1] - len(null_analysis)}")

# Data Quality Validation
print("\n--- Data Quality Validation ---")
validation_rules = {
    'min_rows': 10000,
    'max_missing_pct': 30,
    'required_columns': ['BORO_NM', 'LAW_CAT_CD'],
    'numeric_ranges': {
        'Latitude': (40.4, 41.0),  # Approximate NYC bounds
        'Longitude': (-74.3, -73.7)
    }
}

validation_results = validate_data_quality(df, validation_rules)

if validation_results['errors']:
    print("ERRORS:")
    for error in validation_results['errors']:
        print(f"  {error}")

if validation_results['warnings']:
    print("WARNINGS:")
    for warning in validation_results['warnings']:
        print(f"  {warning}")

if not validation_results['errors'] and not validation_results['warnings']:
    print("All data quality checks passed!")

print("\n1.1.1 Standard Null/NaN Analysis - COMPLETED")


=== ANALYSIS SECTION ===
Starting comprehensive analysis of the dataset...

1.1 COMPREHENSIVE MISSING VALUE ANALYSIS
Performing sophisticated missing value analysis...

Columns with null/NaN values:
   Column  Null_Count  Null_Percentage
    PD_CD        2090         0.083183
 Latitude          28         0.001114
Longitude          28         0.001114

--- Missing Value Strategy Recommendations ---
  PD_CD (0.08% missing): simple_imputation
  Latitude (0.00% missing): simple_imputation
  Longitude (0.00% missing): simple_imputation

--- Top Missing Value Patterns ---
  Pattern: BORO_NM=False, CMPLNT_FR_DT=False, CMPLNT_FR_TM=False, KY_CD=False, LAW_CAT_CD=False, LOC_OF_OCCUR_DESC=False, Latitude=False, Longitude=False, OFNS_DESC=False, PARKS_NM=False, PD_CD=False, PREM_TYP_DESC=False, SUSP_AGE_GROUP=False, SUSP_RACE=False, SUSP_SEX=False, VIC_AGE_GROUP=False, VIC_RACE=False, VIC_SEX=False -> 2510447 rows (99.92%)
  Pattern: BORO_NM=False, CMPLNT_FR_DT=False, CMPLNT_FR_TM=False, KY_CD

### 1.1.2 String Anomaly Analysis

Identify and count occurrences of empty strings, whitespace-only strings, and special character strings in object-type columns.

In [9]:
# 1.1.2 Check for empty strings and other anomalous string values using helper function
print("\n1.1.2 String Anomaly Analysis")
print("Checking for empty strings and other anomalous string values...")

string_anomalies = detect_string_anomalies(df)

# Display results
if string_anomalies['empty_strings']:
    print("\nColumns with empty strings:")
    for col, count in string_anomalies['empty_strings'].items():
        print(f"  {col}: {count} empty strings ({count/len(df)*100:.4f}%)")
else:
    print("\nNo columns with empty strings found.")

if string_anomalies['whitespace_only']:
    print("\nColumns with whitespace-only strings:")
    for col, count in string_anomalies['whitespace_only'].items():
        print(f"  {col}: {count} whitespace-only values ({count/len(df)*100:.4f}%)")
else:
    print("\nNo columns with only whitespace strings found.")

if string_anomalies['special_chars_only']:
    print("\nColumns with special-char-only strings (excluding specific placeholders like '(NULL)'):")
    for col, count in string_anomalies['special_chars_only'].items():
         print(f"  {col}: {count} special-char-only values ({count/len(df)*100:.4f}%)")
else:
    print("\nNo columns with only special characters found.")


1.1.2 String Anomaly Analysis
Checking for empty strings and other anomalous string values...

No columns with empty strings found.

No columns with only whitespace strings found.

No columns with only special characters found.


### 1.1.3 Placeholder Value Analysis

Search for common placeholder strings (e.g., 'UNKNOWN', 'N/A', '-', '9999', '(NULL)') in object columns.

In [10]:
# 1.1.3 Check for placeholder values using helper function
print("\n1.1.3 Placeholder Value Analysis")
print("Checking for common placeholder values that might represent missing data...")

placeholder_counts = detect_placeholder_values(df)

if placeholder_counts:
    print("\nFound potential placeholder values for missing data:")
    for col, counts in placeholder_counts.items():
        if not counts.empty:
            print(f"  {col}:")
            for placeholder, count in counts.items():
                # Try to find original case for display
                original_case_placeholder = placeholder
                try:
                    original_vals = df[col][df[col].astype(str).str.upper() == placeholder].unique()
                    if len(original_vals) == 1:
                        original_case_placeholder = original_vals[0]
                except:
                    pass
                print(f"    '{original_case_placeholder}': {count} occurrences ({count/len(df)*100:.4f}%)")
else:
    print("\nNo common placeholder values found in object columns.")


1.1.3 Placeholder Value Analysis
Checking for common placeholder values that might represent missing data...

Found potential placeholder values for missing data:
  BORO_NM:
    '(null)': 3689 occurrences (0.1468%)
  LOC_OF_OCCUR_DESC:
    '(null)': 470255 occurrences (18.7163%)
  OFNS_DESC:
    '(null)': 44 occurrences (0.0018%)
  PARKS_NM:
    '(null)': 2496688 occurrences (99.3690%)
  PREM_TYP_DESC:
    '(null)': 36876 occurrences (1.4677%)
  SUSP_AGE_GROUP:
    'UNKNOWN': 941287 occurrences (37.4635%)
    '(null)': 412809 occurrences (16.4299%)
  SUSP_RACE:
    'UNKNOWN': 630863 occurrences (25.1086%)
    '(null)': 412809 occurrences (16.4299%)
  SUSP_SEX:
    'U': 526492 occurrences (20.9546%)
    '(null)': 412809 occurrences (16.4299%)
  VIC_AGE_GROUP:
    'UNKNOWN': 715543 occurrences (28.4789%)
    '(null)': 1 occurrences (0.0000%)
  VIC_RACE:
    'UNKNOWN': 767959 occurrences (30.5650%)
    '(null)': 452 occurrences (0.0180%)
  VIC_SEX:
    '(null)': 1 occurrences (0.0000%)


## 1.2 Analyze Age Group Values

Examine the unique values and formats in suspect and victim age group columns to identify any entries that don't match expected patterns.

In [11]:
print("\n1.2 ANALYZING AGE GROUP VALUES")

# Define age group columns to check
age_group_columns = ['SUSP_AGE_GROUP', 'VIC_AGE_GROUP']
print(f"Checking for unusual values in age group columns: {age_group_columns}")

# Define the expected valid age group formats (used for cleaning)
# These patterns help identify values that need fixing or removal
expected_age_formats = [
    r'^\d{1,2}-\d{1,2}$', # e.g., 25-44, 18-24
    r'^<\d{1,2}$',        # e.g., <18
    r'^\d{1,2}\+$',       # e.g., 65+
    r'^UNKNOWN$',         # The standard placeholder eventually used
    r'^\(NULL\)$',        # The placeholder found in the data that needs handling
]
# Compile regex patterns for efficiency (case-insensitive)
compiled_patterns = [re.compile(p, re.IGNORECASE) for p in expected_age_formats]

# Function to check if a value matches any of the expected formats or is NaN
def is_expected_age_format(value):
    if pd.isna(value):
        return True # Treat NaN as 'expected' for analysis (handled in cleaning)
    value_str = str(value).strip()
    for pattern in compiled_patterns:
        if pattern.match(value_str):
            return True
    return False

for col in age_group_columns:
    if col in df.columns:
        print(f"\n--- Analyzing '{col}' ---")
        print(f"  Unique values count: {df[col].nunique()}")

        # Count missing values (NaN)
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            print(f"  Found {nan_count} NaN values ({nan_count/len(df)*100:.2f}%)")

        # Get value counts (excluding NaN for this part)
        value_counts = df[col].value_counts(dropna=True)
        print(f"\n  Top 10 most common non-NaN values:")
        for i, (val, count) in enumerate(value_counts.head(10).items()):
            print(f"    {i+1}. '{val}': {count} occurrences ({count/len(df)*100:.2f}%)")

        # Identify unusual values (those not matching expected formats)
        # This mask selects rows where the value is NOT NaN AND does NOT match expected formats
        unusual_values_mask = ~df[col].apply(is_expected_age_format) & df[col].notna()
        unusual_values = df.loc[unusual_values_mask, col].value_counts()

        if not unusual_values.empty:
            print(f"\n  Found {len(unusual_values)} distinct unusual values (excluding NaN and expected formats like '25-44', '<18', '65+', 'UNKNOWN', '(NULL)' ):")
            # Show top N unusual values
            for i, (val, count) in enumerate(unusual_values.head(20).items()): # Show top 20 unusual
                 print(f"    '{val}': {count} occurrences ({count/len(df)*100:.4f}%)")
            if len(unusual_values) > 20:
                 print(f"    ... and {len(unusual_values) - 20} more distinct unusual values.")
        else:
            print("\n  No unusual age group formats found (all non-NaN values match expected patterns).")

    else:
        print(f"\n  Column '{col}' not found in dataset.")


1.2 ANALYZING AGE GROUP VALUES
Checking for unusual values in age group columns: ['SUSP_AGE_GROUP', 'VIC_AGE_GROUP']

--- Analyzing 'SUSP_AGE_GROUP' ---
  Unique values count: 85

  Top 10 most common non-NaN values:
    1. 'UNKNOWN': 941287 occurrences (37.46%)
    2. '25-44': 676766 occurrences (26.94%)
    3. '(null)': 412809 occurrences (16.43%)
    4. '45-64': 231471 occurrences (9.21%)
    5. '18-24': 178229 occurrences (7.09%)
    6. '<18': 46631 occurrences (1.86%)
    7. '65+': 25131 occurrences (1.00%)
    8. '1022': 25 occurrences (0.00%)
    9. '1023': 20 occurrences (0.00%)
    10. '2021': 19 occurrences (0.00%)

  Found 78 distinct unusual values (excluding NaN and expected formats like '25-44', '<18', '65+', 'UNKNOWN', '(NULL)' ):
    '1022': 25 occurrences (0.0010%)
    '1023': 20 occurrences (0.0008%)
    '2021': 19 occurrences (0.0008%)
    '2022': 17 occurrences (0.0007%)
    '2023': 13 occurrences (0.0005%)
    '2024': 13 occurrences (0.0005%)
    '2020': 10 occurr

## 1.3 Check for Duplicate Records

Identify and count exact duplicate rows across all columns.

In [12]:
# 1.3 Checking for duplicate records
print("\n1.3 CHECKING FOR DUPLICATE RECORDS")

# Check for exact duplicates across all columns
exact_duplicates_count = df.duplicated().sum()
if exact_duplicates_count > 0:
    print(f"Found {exact_duplicates_count} exact duplicate rows ({exact_duplicates_count/len(df)*100:.4f}% of data)")
else:
    print("No exact duplicate rows found.")


1.3 CHECKING FOR DUPLICATE RECORDS
Found 9502 exact duplicate rows (0.3782% of data)


## 1.4 Analyze Categorical Value Standardization Needs

Check specific categorical columns for inconsistent representations and case variations that require standardization.

In [13]:
# 1.4 Analyzing standardization needs for categorical values
print("\n1.4 ANALYZING CATEGORICAL VALUE STANDARDIZATION NEEDS")

# Columns to check for potential inconsistencies and variations
categorical_cols_to_analyze = [
    'BORO_NM', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC',
    'PARKS_NM', 'PREM_TYP_DESC', 'SUSP_RACE', 'SUSP_SEX',
    'VIC_RACE', 'VIC_SEX'
]

standardization_suggestions = {}

for col in categorical_cols_to_analyze:
    # Check if column exists and is of object type (likely categorical string)
    if col in df.columns and pd.api.types.is_object_dtype(df[col]):
        print(f"\n--- Analyzing '{col}' ---")
        # Use .astype(str) to handle potential mixed types safely
        unique_values = df[col].astype(str).unique() # Get unique values including potential 'nan' string
        unique_values_non_nan_str = df[col].dropna().astype(str).unique()
        num_unique = len(unique_values_non_nan_str)
        print(f"  Number of unique non-NaN string values: {num_unique}")

        # Limit display for very high cardinality columns
        if num_unique > 100:
             print("  (Too many unique values to list sample)")
        elif num_unique > 0:
            sample = list(unique_values_non_nan_str)[:15]
            print(f"  Sample unique values: {sample}{'...' if num_unique > 15 else ''}")
        else:
             print("  No unique non-NaN string values found.")

        # Check for case inconsistencies (e.g., 'Brooklyn' vs 'BROOKLYN')
        if num_unique > 0:
            try:
                lower_unique_count = df[col].dropna().astype(str).str.lower().nunique()
                upper_unique_count = df[col].dropna().astype(str).str.upper().nunique()
                if lower_unique_count != num_unique or upper_unique_count != num_unique:
                    print(f"  ! Found case inconsistencies (e.g., potentially '{unique_values_non_nan_str[0]}' vs '{unique_values_non_nan_str[0].upper()}'). Suggest converting to uniform case (e.g., UPPER).")
                    if col not in standardization_suggestions:
                        standardization_suggestions[col] = []
                    if 'Convert to UPPERCASE' not in standardization_suggestions[col]:
                         standardization_suggestions[col].append('Convert to UPPERCASE')

            except Exception as e:
                print(f"  Warning: Could not perform case check on {col}: {e}")

            # Check for leading/trailing whitespace
            try:
                # Check if any value is different after stripping
                has_whitespace = (df[col].dropna().astype(str) != df[col].dropna().astype(str).str.strip()).any()
                if has_whitespace:
                    print("  ! Found values with leading/trailing whitespace. Suggest stripping whitespace.")
                    if col not in standardization_suggestions:
                        standardization_suggestions[col] = []
                    if 'Strip whitespace' not in standardization_suggestions[col]:
                         standardization_suggestions[col].append('Strip whitespace')
            except Exception as e:
                 print(f"  Warning: Could not perform whitespace check on {col}: {e}")

        # Specific checks based on column knowledge (using uppercase for comparison)
        col_upper_series = df[col].dropna().astype(str).str.upper()

        if col == 'BORO_NM':
            abbreviations = {'MN', 'M', 'BX', 'BK', 'K', 'QN', 'Q', 'SI', 'R'}
            found_abbr = abbreviations.intersection(set(col_upper_series.unique()))
            if found_abbr:
                print(f"  ! Found potential borough abbreviations like {found_abbr}. Suggest mapping to full names (e.g., MN -> MANHATTAN).")
                if col not in standardization_suggestions:
                     standardization_suggestions[col] = []
                if 'Map abbreviations to full names' not in standardization_suggestions[col]:
                     standardization_suggestions[col].append('Map abbreviations to full names')

        elif col == 'LOC_OF_OCCUR_DESC':
            variations = {'FRONT OF', 'OPPOSITE OF', 'REAR OF'}
            found_vars = variations.intersection(set(col_upper_series.unique()))
            if found_vars:
                print(f"  ! Found location variations like {found_vars}. Suggest standardizing (e.g., FRONT OF -> FRONT).")
                if col not in standardization_suggestions:
                     standardization_suggestions[col] = []
                if 'Standardize location descriptions' not in standardization_suggestions[col]:
                     standardization_suggestions[col].append('Standardize location descriptions')

        elif col in ['SUSP_SEX', 'VIC_SEX']:
             # Expected values after cleaning placeholders ('(NULL)', 'U') should be M, F, U
             expected_sex = {'M', 'F', 'U'}
             # Check for values that are NOT M, F, U, AND also not the known placeholders we handle separately
             known_placeholders_upper = {'UNKNOWN', '(NULL)'} # Placeholders handled in other steps
             current_unique_upper = set(col_upper_series.unique())
             unexpected_sex = current_unique_upper - expected_sex - known_placeholders_upper

             if unexpected_sex:
                 print(f"  ! Found unexpected sex values like {unexpected_sex} (excluding known placeholders). Suggest mapping to M, F, or U/UNKNOWN.")
                 if col not in standardization_suggestions:
                     standardization_suggestions[col] = []
                 if 'Map unexpected values to M/F/U' not in standardization_suggestions[col]:
                     standardization_suggestions[col].append('Map unexpected values to M/F/U')

    elif col in df.columns:
         print(f"\n--- Skipping '{col}' (Not object type: {df[col].dtype}) ---")
    # else: Column not found - implicitly skipped


print("\n--- Summary of Standardization Suggestions ---")
if standardization_suggestions:
    for col, suggestions in standardization_suggestions.items():
        print(f"  {col}: {', '.join(suggestions)}")
else:
    print("No obvious standardization needs detected based on these checks.")


1.4 ANALYZING CATEGORICAL VALUE STANDARDIZATION NEEDS

--- Analyzing 'BORO_NM' ---
  Number of unique non-NaN string values: 6
  Sample unique values: ['BRONX', 'QUEENS', 'BROOKLYN', 'MANHATTAN', 'STATEN ISLAND', '(null)']

--- Analyzing 'LAW_CAT_CD' ---
  Number of unique non-NaN string values: 3
  Sample unique values: ['MISDEMEANOR', 'FELONY', 'VIOLATION']

--- Analyzing 'LOC_OF_OCCUR_DESC' ---
  Number of unique non-NaN string values: 6
  Sample unique values: ['INSIDE', 'FRONT OF', '(null)', 'REAR OF', 'OPPOSITE OF', 'OUTSIDE']
  ! Found location variations like {'FRONT OF', 'REAR OF', 'OPPOSITE OF'}. Suggest standardizing (e.g., FRONT OF -> FRONT).

--- Analyzing 'OFNS_DESC' ---
  Number of unique non-NaN string values: 69
  Sample unique values: ['PETIT LARCENY', 'CRIMINAL MISCHIEF & RELATED OF', 'FELONY ASSAULT', 'FORGERY', 'HARRASSMENT 2', 'ASSAULT 3 & RELATED OFFENSES', 'BURGLARY', 'ROBBERY', 'FRAUDS', 'DANGEROUS WEAPONS', 'MISCELLANEOUS PENAL LAW', 'OFF. AGNST PUB ORD SENSB

## 1.5 Validating Dates and Times Formats

Detect columns whose names suggest dates or times, attempt to parse each value, and report counts and examples of any invalid entries.

In [14]:
print("\n1.5 VALIDATING DATES AND TIMES FORMATS")

# Identify potential date and time columns based on naming conventions
date_columns = [col for col in df.columns if any(term in col.upper() for term in ['DATE', 'DT'])]
time_columns = [col for col in df.columns if any(term in col.upper() for term in ['TIME', 'TM'])]

print(f"Identified {len(date_columns)} potential date columns: {date_columns}")
print(f"Identified {len(time_columns)} potential time columns: {time_columns}")

# Validate Date Columns
for col in date_columns:
    if col in df.columns:
        print(f"\n--- Validating Date Format in '{col}' ---")
        # Attempt conversion to datetime, coercing errors to NaT (Not a Time)
        # Assuming common formats; might need refinement if specific formats are known (e.g., format='%m/%d/%Y')
        try:
            # Make a copy to avoid modifying the original DataFrame during analysis
            dates_series = df[col].copy()
            # Attempt conversion - this is the most crucial step
            # Using infer_datetime_format=True might speed up parsing if formats are consistent
            parsed_dates = pd.to_datetime(dates_series, errors='coerce', infer_datetime_format=True)

            # Check for invalid dates (NaT values resulting from parsing errors)
            invalid_dates_mask = parsed_dates.isna() & df[col].notna() # Focus on non-NaN originals that failed parsing
            invalid_count = invalid_dates_mask.sum()

            original_nan_count = df[col].isna().sum()
            total_unparseable = original_nan_count + invalid_count

            if total_unparseable > 0:
                 print(f"Found {total_unparseable} total unparseable/missing date entries ({total_unparseable/len(df)*100:.2f}%)." )
                 if original_nan_count > 0:
                      print(f"({original_nan_count} were originally NaN/missing).")
                 if invalid_count > 0:
                      print(f"    ({invalid_count} failed parsing due to format issues)." )
                      # Show examples of the original values that failed parsing
                      invalid_examples = df.loc[invalid_dates_mask, col].unique() # Show unique original values that failed
                      print(f"    Examples of values that failed parsing: {list(invalid_examples)[:10]}{'...' if len(invalid_examples) > 10 else ''}")
            else:
                print("  All values seem to be either NaN or in a parseable date format.")

            # Show date range for valid dates
            valid_dates = parsed_dates.dropna()
            if not valid_dates.empty:
                print(f"  Date range for parseable dates: {valid_dates.min().strftime('%Y-%m-%d')} to {valid_dates.max().strftime('%Y-%m-%d')}")
            elif total_unparseable < len(df):
                 # This case should ideally not happen if the above logic is correct
                 print("  Could not determine date range, although some valid dates might exist.")
            else:
                 print("  No valid dates found to determine a range.")

        except Exception as e:
            print(f"  ERROR validating dates in '{col}': {e}. Check column data type and content.")

# Validate Time Columns
for col in time_columns:
    if col in df.columns:
        print(f"\n--- Validating Time Format in '{col}' ---")
        try:
            # Make a copy to avoid modifying the original DataFrame
            time_series = df[col].copy()
            # Handle potential non-string types before conversion
            time_series_str = time_series.astype(str)

            # Attempt conversion to time objects, coercing errors
            # Using format='%H:%M:%S' - adjust if needed
            parsed_times = pd.to_datetime(time_series_str, format='%H:%M:%S', errors='coerce').dt.time

            # Check for invalid times (NaT values resulting from parsing errors)
            invalid_times_mask = parsed_times.isna() & df[col].notna() # Focus on non-NaN originals that failed
            invalid_count = invalid_times_mask.sum()

            original_nan_count = df[col].isna().sum()
            total_unparseable = original_nan_count + invalid_count

            if total_unparseable > 0:
                print(f"  Found {total_unparseable} total unparseable/missing time entries ({total_unparseable/len(df)*100:.2f}%)." )
                if original_nan_count > 0:
                     print(f"    ({original_nan_count} were originally NaN/missing).")
                if invalid_count > 0:
                     print(f"    ({invalid_count} failed parsing due to format issues)." )
                     # Show examples of original values that failed parsing
                     invalid_examples = df.loc[invalid_times_mask, col].unique()
                     print(f"    Examples of values that failed parsing: {list(invalid_examples)[:10]}{'...' if len(invalid_examples) > 10 else ''}")
            else:
                print("  All values seem to be either NaN or in a parseable time format (HH:MM:SS).")

            # Show time range for valid times
            valid_times = parsed_times.dropna()
            if not valid_times.empty:
                # Convert times to seconds past midnight for robust comparison
                def time_to_seconds(t):
                   if pd.isna(t): return -1 # Ensure NaNs sort last/first depending on need
                   try:
                       return t.hour * 3600 + t.minute * 60 + t.second
                   except AttributeError:
                       return -1 # Handle unexpected types

                # Filter out potential -1 values if any conversion errors occurred
                valid_seconds = [s for s in (time_to_seconds(t) for t in valid_times) if s >= 0]
                if valid_seconds:
                     min_time_obj = min(valid_times, key=time_to_seconds)
                     max_time_obj = max(valid_times, key=time_to_seconds)
                     print(f"  Time range for parseable times: {min_time_obj.strftime('%H:%M:%S')} to {max_time_obj.strftime('%H:%M:%S')}")
                else:
                     print("  Could not determine time range from valid times (possibly conversion issues).")
            elif total_unparseable < len(df):
                 print("  Could not determine time range, although some valid times might exist.")
            else:
                 print("  No valid times found to determine a range.")

        except Exception as e:
            print(f"  ERROR validating times in '{col}': {e}. Check column data type and content. Expected format HH:MM:SS.")


1.5 VALIDATING DATES AND TIMES FORMATS
Identified 1 potential date columns: ['CMPLNT_FR_DT']
Identified 1 potential time columns: ['CMPLNT_FR_TM']

--- Validating Date Format in 'CMPLNT_FR_DT' ---
  All values seem to be either NaN or in a parseable date format.
  Date range for parseable dates: 2020-01-01 to 2024-12-31

--- Validating Time Format in 'CMPLNT_FR_TM' ---
  All values seem to be either NaN or in a parseable time format (HH:MM:SS).
  Time range for parseable times: 00:00:00 to 23:59:00


# Cleaning

Apply deterministic, auditable rules to make the dataset robust for integration and modeling.

## 2.0 Initial Setup for Cleaning

Record the original number of rows and create a dictionary to track rows removed by each cleaning step.

In [ ]:
print("\n=== CLEANING SECTION ===")
print("Starting cleaning operations on the dataset...")

# Store the original row count for final comparison
# Ensure 'df' exists and has data before getting length
if 'df' in locals() and isinstance(df, pd.DataFrame):
    original_row_count = len(df)
    print(f"Original number of rows: {original_row_count}")
else:
    print("Error: DataFrame 'df' not found or not loaded correctly. Cannot proceed with cleaning.")
    # Handle error appropriately, maybe raise an exception or exit
    original_row_count = 0 # Set to 0 to avoid errors later, but indicate failure

# Dictionary to track rows removed during cleaning steps
cleaning_rows_removed = {}

# Apply missing value strategy
print("\n2.0 MISSING VALUE ANALYSIS AND PLANNING")
if original_row_count > 0:
    # Get missing value strategy
    missing_strategies = missing_value_strategy(df)

    print("\n--- Missing Value Strategy Plan ---")
    strategy_counts = {}
    for col, strategy in missing_strategies.items():
        strategy_counts[strategy] = strategy_counts.get(strategy, 0) + 1
        missing_pct = (df[col].isna().sum() / len(df)) * 100
        if missing_pct > 0:
            print(f"  {col} ({missing_pct:.1f}% missing): {strategy}")

    print("\n--- Strategy Summary ---")
    for strategy, count in strategy_counts.items():
        print(f"  {strategy}: {count} columns")

    # Apply drop_column strategy first
    columns_to_drop = [col for col, strategy in missing_strategies.items() if strategy == 'drop_column']
    if columns_to_drop:
        print(f"\n--- Dropping High-Missingness Columns ---")
        print(f"Columns to drop: {columns_to_drop}")
        original_cols = len(df.columns)
        df = df.drop(columns=columns_to_drop)
        print(f"Dropped {len(columns_to_drop)} columns. Remaining: {len(df.columns)}")
        cleaning_rows_removed['Dropped_Columns'] = columns_to_drop

    # Validate data quality after initial cleaning
    print("\n--- Post-Strategy Data Quality Check ---")
    validation_results = validate_data_quality(df)

    if validation_results['errors']:
        print("CRITICAL ERRORS FOUND:")
        for error in validation_results['errors']:
            print(f"  ERROR: {error}")
            

    if validation_results['warnings']:
        print("WARNINGS:")
        for warning in validation_results['warnings']:
            print(f"  WARNING: {warning}")

    if not validation_results['errors'] and not validation_results['warnings']:
        print("All data quality checks passed after initial cleaning!")
else:
    print("Skipping missing value analysis due to DataFrame loading issues.")


=== CLEANING SECTION ===
Starting cleaning operations on the dataset...
Original number of rows: 2512541

2.0 SOPHISTICATED MISSING VALUE ANALYSIS AND PLANNING

--- Missing Value Strategy Plan ---
  Latitude (0.0% missing): simple_imputation
  Longitude (0.0% missing): simple_imputation
  PD_CD (0.1% missing): simple_imputation

--- Strategy Summary ---
  no_action: 15 columns
  simple_imputation: 3 columns

--- Post-Strategy Data Quality Check ---
🟡 WARNINGS:


## 2.1 Clean: Handle Missing Geographic Coordinates with Stratified Sampling

Remove rows with missing coordinates while maintaining the distribution of other important variables.

In [16]:
print("\n2.1 CLEANING GEOGRAPHIC COORDINATES DATA WITH STRATIFIED SAMPLING")

if original_row_count > 0:
    # Identify latitude and longitude columns
    lat_col = next((col for col in df.columns if 'LATITUDE' in col.upper()), None)
    lon_col = next((col for col in df.columns if 'LONGITUDE' in col.upper()), None)

    if lat_col and lon_col:
        rows_before_geo = len(df)
        initial_na_lat = df[lat_col].isna().sum()
        initial_na_lon = df[lon_col].isna().sum()
        print(f"Initial missing values - {lat_col}: {initial_na_lat}, {lon_col}: {initial_na_lon}")

        # Create mask for rows with missing coordinates
        missing_coords_mask = df[lat_col].isna() | df[lon_col].isna()
        missing_coords_count = missing_coords_mask.sum()

        if missing_coords_count > 0:
            print(f"Found {missing_coords_count} rows with missing coordinates ({missing_coords_count/len(df)*100:.2f}%)")

            # Use stratified removal if we have a suitable stratification column
            stratify_col = None
            for potential_col in ['BORO_NM', 'LAW_CAT_CD', 'OFNS_DESC']:
                if potential_col in df.columns and df[potential_col].notna().sum() > 0:
                    stratify_col = potential_col
                    break

            if stratify_col and missing_coords_count / len(df) > 0.05:  # Use stratified if >5% missing
                print(f"Using stratified removal based on '{stratify_col}' to maintain distribution...")

                # Show distribution before removal
                before_dist = df[stratify_col].value_counts(normalize=True)
                print("Distribution before removal:")
                for val, prop in before_dist.head().items():
                    print(f"  {val}: {prop:.3f}")

                # Apply stratified removal
                df = stratified_data_removal(df, stratify_col, missing_coords_mask, max_removal_pct=0.15)

                # Show distribution after removal
                after_dist = df[stratify_col].value_counts(normalize=True)
                print("Distribution after removal:")
                for val, prop in after_dist.head().items():
                    print(f"  {val}: {prop:.3f}")

            else:
                # Simple removal for small amounts of missing data
                print("Using simple removal (missing data percentage is low)...")
                df = df.dropna(subset=[lat_col, lon_col])

        rows_after_geo = len(df)
        rows_removed = rows_before_geo - rows_after_geo
        cleaning_rows_removed['Missing Geographic Coordinates'] = rows_removed
        print(f"Removed {rows_removed} rows with missing coordinates.")
        print(f"Rows remaining: {rows_after_geo}")
    else:
        print("Warning: Latitude or Longitude columns not found. Skipping geographic coordinate cleaning.")
        cleaning_rows_removed['Missing Geographic Coordinates'] = 0
else:
    print("Skipping step 2.1 due to DataFrame loading issues.")


2.1 CLEANING GEOGRAPHIC COORDINATES DATA WITH STRATIFIED SAMPLING
Initial missing values - Latitude: 28, Longitude: 28
Found 28 rows with missing coordinates (0.00%)
Using simple removal (missing data percentage is low)...
Removed 28 rows with missing coordinates.
Rows remaining: 2512513


## 2.2 Clean: Handle Missing PD_CD Values

Remove rows where the `PD_CD` column has a NaN value.

In [17]:
print("\n2.2 CLEANING PD_CD VALUES")

if original_row_count > 0: # Proceed only if df was loaded
    pd_cd_col = 'PD_CD' # Assuming this is the correct column name

    if pd_cd_col in df.columns:
        rows_before_pdcd = len(df)
        initial_na_pdcd = df[pd_cd_col].isna().sum()
        print(f"Initial missing values in {pd_cd_col}: {initial_na_pdcd}")

        # Drop rows where PD_CD is missing
        df = df.dropna(subset=[pd_cd_col])
        rows_after_pdcd = len(df)
        rows_removed = rows_before_pdcd - rows_after_pdcd

        # Store the count of removed rows
        cleaning_rows_removed['Missing PD_CD'] = rows_removed
        print(f"Dropped {rows_removed} rows with missing {pd_cd_col} values.")
        print(f"Rows remaining: {rows_after_pdcd}")
    else:
        print(f"Warning: Column '{pd_cd_col}' not found. Skipping PD_CD cleaning.")
        cleaning_rows_removed['Missing PD_CD'] = 0
else:
    print("Skipping step 2.2 due to DataFrame loading issues.")


2.2 CLEANING PD_CD VALUES
Initial missing values in PD_CD: 2066
Dropped 2066 rows with missing PD_CD values.
Rows remaining: 2510447


## 2.3 Clean: Handle Placeholder Values in Specific Columns

Remove rows where `BORO_NM` contains the value `'(NULL)'` (case-insensitive).

In [18]:
print("\n2.3 CLEANING SPECIFIC PLACEHOLDER VALUES")

if original_row_count > 0: # Proceed only if df was loaded
    placeholder_to_remove = '(NULL)'
    columns_to_clean = {'BORO_NM'} # Set for efficient lookup
    total_placeholder_rows_removed = 0

    for col in columns_to_clean:
        if col in df.columns:
            rows_before_placeholder = len(df)
            # Check for placeholder case-insensitively
            try:
                # Use .astype(str) to safely handle potential non-string data before comparison
                is_placeholder_mask = df[col].astype(str).str.upper() == placeholder_to_remove.upper()
                placeholder_count = is_placeholder_mask.sum()
                print(f"Found {placeholder_count} rows with '{placeholder_to_remove}' (case-insensitive) in {col}.")

                if placeholder_count > 0:
                    # Remove rows with the placeholder
                    df = df[~is_placeholder_mask]
                    rows_after_placeholder = len(df)
                    rows_removed = rows_before_placeholder - rows_after_placeholder
                    # Use a more specific key for tracking
                    removal_key = f'Placeholder {placeholder_to_remove} in {col}'
                    cleaning_rows_removed[removal_key] = rows_removed
                    total_placeholder_rows_removed += rows_removed
                    print(f"Removed {rows_removed} rows with '{placeholder_to_remove}' in {col}.")
                else:
                     # Ensure the key exists even if 0 rows were removed
                     removal_key = f'Placeholder {placeholder_to_remove} in {col}'
                     cleaning_rows_removed[removal_key] = 0
            except Exception as e:
                print(f"Warning: Could not clean placeholder '{placeholder_to_remove}' in column {col}: {e}")
                removal_key = f'Placeholder {placeholder_to_remove} in {col}'
                cleaning_rows_removed[removal_key] = 0 # Record 0 removals on error
        else:
            print(f"Warning: Column '{col}' not found for placeholder cleaning.")
            removal_key = f'Placeholder {placeholder_to_remove} in {col}'
            cleaning_rows_removed[removal_key] = 0 # Record 0 removals if column missing

    print(f"Total rows removed due to '{placeholder_to_remove}' in specified columns: {total_placeholder_rows_removed}")
    print(f"Rows remaining: {len(df)}")
else:
    print("Skipping step 2.3 due to DataFrame loading issues.")


2.3 CLEANING SPECIFIC PLACEHOLDER VALUES
Found 3689 rows with '(NULL)' (case-insensitive) in BORO_NM.
Removed 3689 rows with '(NULL)' in BORO_NM.
Total rows removed due to '(NULL)' in specified columns: 3689
Rows remaining: 2506758


## 2.4 Clean: Fill or Standardize Missing/Placeholder Values

Replace missing values or placeholders with standardized values in specified columns.

In [19]:
print("\n2.4 FILLING AND STANDARDIZING MISSING/PLACEHOLDER VALUES")

if original_row_count > 0: # Proceed only if df was loaded
    # Define columns and their fill/standardization strategy
    fill_strategy = {
        'LOC_OF_OCCUR_DESC': 'UNKNOWN', # Fill NaN only initially, (NULL) handled later if needed
        'PREM_TYP_DESC': 'UNKNOWN',   # Fill NaN and specific placeholders
        'SUSP_AGE_GROUP': 'UNKNOWN',  # Fill NaN and specific placeholders
        'SUSP_RACE': 'UNKNOWN',     # Fill NaN and specific placeholders
        'SUSP_SEX': 'U',            # Fill NaN and specific placeholders
        'VIC_RACE': 'UNKNOWN'       # Fill specific placeholders only initially (NaN handled later)
        # 'VIC_AGE_GROUP' has (NULL) handled separately below, NaN handled later
        # 'PARKS_NM' is intentionally skipped
    }

    # Placeholders to target for replacement (case-insensitive)
    placeholders_to_replace = ['(NULL)', 'UNKNOWN']
    placeholders_to_replace_upper = [p.upper() for p in placeholders_to_replace]

    for col, fill_value in fill_strategy.items():
        if col in df.columns:
            print(f"--- Processing '{col}' (Fill/Standardize with '{fill_value}') ---")
            nan_before = df[col].isna().sum()
            placeholders_replaced_count = 0

            # --- Apply Fill/Standardization ---
            # 1. Fill NaN values first
            df[col] = df[col].fillna(fill_value)
            nan_filled = nan_before # Assume all NaNs are filled
            if nan_filled > 0:
                print(f"  Filled {nan_filled} NaN values with '{fill_value}'.")

            # 2. Replace specific placeholders (case-insensitive)
            # Ensure the column is string type for accurate comparison
            try:
                col_str_upper = df[col].astype(str).str.upper()
                # Create a mask for all specified placeholders
                mask_is_placeholder = col_str_upper.isin(placeholders_to_replace_upper)
                # Exclude cases where the placeholder IS the intended fill value
                mask_is_not_fill_value = col_str_upper != fill_value.upper()

                # Combine masks: Is a placeholder AND is not the target fill value
                mask_to_replace = mask_is_placeholder & mask_is_not_fill_value

                placeholders_replaced_count = mask_to_replace.sum()
                if placeholders_replaced_count > 0:
                     df.loc[mask_to_replace, col] = fill_value
                     print(f"  Standardized {placeholders_replaced_count} placeholder values (e.g., {placeholders_to_replace}) to '{fill_value}'.")
            except Exception as e:
                print(f"  Warning: Could not replace placeholders in {col}: {e}")

        else:
            print(f"Warning: Column '{col}' not found for filling/standardizing.")

    # Special handling for (NULL) in VIC_AGE_GROUP if not covered above
    vic_age_col = 'VIC_AGE_GROUP'
    if vic_age_col in df.columns and vic_age_col not in fill_strategy:
         try:
            # Ensure string type for comparison
            vic_age_null_mask = df[vic_age_col].astype(str).str.upper() == '(NULL)'
            vic_age_null_count = vic_age_null_mask.sum()
            if vic_age_null_count > 0:
                 df.loc[vic_age_null_mask, vic_age_col] = 'UNKNOWN' # Standardize to UNKNOWN
                 print(f"--- Processing '{vic_age_col}' ---")
                 print(f"  Standardized {vic_age_null_count} '(NULL)' values to 'UNKNOWN'.")
         except Exception as e:
              print(f"Warning: Could not replace (NULL) in {vic_age_col}: {e}")

    print("Finished filling and standardizing missing/placeholder values.")
    print(f"Rows remaining: {len(df)}") # Row count should not change here
else:
    print("Skipping step 2.4 due to DataFrame loading issues.")


2.4 FILLING AND STANDARDIZING MISSING/PLACEHOLDER VALUES
--- Processing 'LOC_OF_OCCUR_DESC' (Fill/Standardize with 'UNKNOWN') ---
  Standardized 466658 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'UNKNOWN'.
--- Processing 'PREM_TYP_DESC' (Fill/Standardize with 'UNKNOWN') ---
  Standardized 35745 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'UNKNOWN'.
--- Processing 'SUSP_AGE_GROUP' (Fill/Standardize with 'UNKNOWN') ---
  Standardized 408893 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'UNKNOWN'.
--- Processing 'SUSP_RACE' (Fill/Standardize with 'UNKNOWN') ---
  Standardized 408893 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'UNKNOWN'.
--- Processing 'SUSP_SEX' (Fill/Standardize with 'U') ---
  Standardized 408893 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'U'.
--- Processing 'VIC_RACE' (Fill/Standardize with 'UNKNOWN') ---
  Standardized 331 placeholder values (e.g., ['(NULL)', 'UNKNOWN']) to 'UNKNOWN'.
--- Processing 'VIC_AGE_GROUP' ---
  Standa

## 2.5 Clean: Handle Missing Victim Information

Remove rows with missing victim information or with `'(NULL)'` in `VIC_SEX`.

In [20]:
print("\n2.5 CLEANING MISSING VICTIM DATA")

if original_row_count > 0: # Proceed only if df was loaded
    rows_before_victim_clean = len(df)
    total_victim_rows_removed = 0

    # Step 1: Remove rows with '(NULL)' in VIC_SEX (as a safety check)
    vic_sex_col = 'VIC_SEX'
    placeholder_vic_sex_key = 'Placeholder (NULL) in VIC_SEX'
    cleaning_rows_removed[placeholder_vic_sex_key] = 0 # Initialize
    if vic_sex_col in df.columns:
        try:
            vic_sex_null_mask = df[vic_sex_col].astype(str).str.upper() == '(NULL)'
            vic_sex_null_count = vic_sex_null_mask.sum()
            if vic_sex_null_count > 0:
                df = df[~vic_sex_null_mask]
                rows_removed_sex_null = rows_before_victim_clean - len(df)
                cleaning_rows_removed[placeholder_vic_sex_key] = rows_removed_sex_null
                total_victim_rows_removed += rows_removed_sex_null
                print(f"Removed {rows_removed_sex_null} rows with '(NULL)' in {vic_sex_col}.")
        except Exception as e:
            print(f"Warning: Could not clean (NULL) in {vic_sex_col}: {e}")

    # Step 2: Remove rows with missing victim age, race, or sex (NaN values)
    vic_columns = ['VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']
    vic_columns_in_df = [col for col in vic_columns if col in df.columns]
    missing_victim_nan_key = 'Missing Victim Info (NaN)'
    cleaning_rows_removed[missing_victim_nan_key] = 0 # Initialize

    if vic_columns_in_df:
        rows_before_nan_removal = len(df)
        # Calculate how many rows have at least one NaN in these columns
        initial_na_mask = df[vic_columns_in_df].isna().any(axis=1)
        initial_na_count = initial_na_mask.sum()
        print(f"Initial count of rows with NaN in any of {vic_columns_in_df}: {initial_na_count}")

        # Drop rows where any of these columns are NaN
        df = df.dropna(subset=vic_columns_in_df)
        rows_after_nan_removal = len(df)
        rows_removed_nan = rows_before_nan_removal - rows_after_nan_removal

        cleaning_rows_removed[missing_victim_nan_key] = rows_removed_nan
        total_victim_rows_removed += rows_removed_nan # Add to total removed in this step
        print(f"Removed {rows_removed_nan} rows with missing (NaN) victim age group, race, or sex.")
    else:
        print(f"Warning: One or more standard victim columns {vic_columns} not found. Skipping NaN removal for victim info.")

    print(f"Total rows removed during victim data cleaning (step 2.5): {cleaning_rows_removed[placeholder_vic_sex_key] + cleaning_rows_removed[missing_victim_nan_key]}")
    print(f"Rows remaining: {len(df)}")
else:
    print("Skipping step 2.5 due to DataFrame loading issues.")


2.5 CLEANING MISSING VICTIM DATA
Removed 1 rows with '(NULL)' in VIC_SEX.
Initial count of rows with NaN in any of ['VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']: 0
Removed 0 rows with missing (NaN) victim age group, race, or sex.
Total rows removed during victim data cleaning (step 2.5): 1
Rows remaining: 2506757


## 2.6 Clean: Handle Remaining General `(NULL)` Values

Replace any remaining occurrences of the `'(NULL)'` placeholder in object-type columns with `'UNKNOWN'`.

In [21]:
print("\n2.6 HANDLING REMAINING GENERAL '(NULL)' VALUES")

if original_row_count > 0: # Proceed only if df was loaded
    placeholder_to_replace = '(NULL)'
    replacement_value = 'UNKNOWN'
    # Define columns already handled or intentionally skipped
    skipped_cols = {'SUSP_SEX', 'PARKS_NM', 'BORO_NM', 'OFNS_DESC', 'VIC_SEX'} # Add cols where (NULL) was removed
    total_general_null_replaced = 0

    object_columns = df.select_dtypes(include=['object']).columns

    for col in object_columns:
        if col not in skipped_cols:
            try:
                # Check for the placeholder case-insensitively
                null_mask = df[col].astype(str).str.upper() == placeholder_to_replace.upper()
                null_count = null_mask.sum()

                if null_count > 0:
                    df.loc[null_mask, col] = replacement_value
                    total_general_null_replaced += null_count
                    print(f"Replaced {null_count} '{placeholder_to_replace}' values in '{col}' with '{replacement_value}'.")

            except Exception as e:
                print(f"Warning: Could not process column {col} for general '{placeholder_to_replace}' replacement: {e}")

    if total_general_null_replaced > 0:
         print(f"Total general '{placeholder_to_replace}' values replaced in this step: {total_general_null_replaced}")
    else:
         print(f"No general '{placeholder_to_replace}' values found requiring replacement in applicable object columns.")

    print(f"Rows remaining: {len(df)}") # Row count should not change here
else:
    print("Skipping step 2.6 due to DataFrame loading issues.")


2.6 HANDLING REMAINING GENERAL '(NULL)' VALUES
No general '(NULL)' values found requiring replacement in applicable object columns.
Rows remaining: 2506757


## 2.7 Clean: Handle Invalid Age Group Formats

Remove rows containing invalid or corrupt age group values.

In [22]:
print("\n2.7 CLEANING INVALID AGE GROUP FORMATS")

if original_row_count > 0: # Proceed only if df was loaded
    # Define the final set of valid patterns
    # '(NULL)' should have been converted to 'UNKNOWN' already.
    valid_age_patterns_final = [
        re.compile(r'^\d{1,2}-\d{1,2}$'), # e.g., 25-44
        re.compile(r'^<\d{1,2}$'),        # e.g., <18
        re.compile(r'^\d{1,2}\+$'),       # e.g., 65+
        re.compile(r'^UNKNOWN$', re.IGNORECASE) # Allow UNKNOWN (case-insensitive)
    ]

    # Function to check if a value is a valid age format (NaNs are considered valid here as they were handled)
    def is_valid_age_format_final(value):
        if pd.isna(value):
            return True # NaNs were handled earlier
        value_str = str(value).strip()
        for pattern in valid_age_patterns_final:
            if pattern.match(value_str):
                return True
        return False

    age_cols_to_validate = ['SUSP_AGE_GROUP', 'VIC_AGE_GROUP']
    total_age_format_rows_removed = 0

    for col in age_cols_to_validate:
        # Define a unique key for tracking removals for this column
        removal_key = f'Invalid Format {col}'
        cleaning_rows_removed[removal_key] = 0 # Initialize

        if col in df.columns:
            rows_before_age_clean = len(df)
            # Identify invalid formats (excluding NaN)
            invalid_age_mask = ~df[col].apply(is_valid_age_format_final) & df[col].notna()
            invalid_count = invalid_age_mask.sum()

            if invalid_count > 0:
                print(f"Found {invalid_count} invalid age formats in '{col}'.")
                # Display samples of invalid values being removed
                invalid_examples = df.loc[invalid_age_mask, col].unique()
                print(f"  Examples being removed: {list(invalid_examples)[:10]}{'...' if len(invalid_examples) > 10 else ''}")

                # Remove rows with invalid formats
                df = df[~invalid_age_mask]
                rows_removed = rows_before_age_clean - len(df)
                cleaning_rows_removed[removal_key] = rows_removed
                total_age_format_rows_removed += rows_removed
                print(f"Removed {rows_removed} rows with invalid age formats in '{col}'.")
            else:
                print(f"No invalid age formats found requiring removal in '{col}'.")
        else:
             print(f"Warning: Column '{col}' not found for age format cleaning.")

    print(f"Total rows removed due to invalid age formats: {total_age_format_rows_removed}")
    print(f"Rows remaining: {len(df)}")
else:
    print("Skipping step 2.7 due to DataFrame loading issues.")


2.7 CLEANING INVALID AGE GROUP FORMATS
Found 215 invalid age formats in 'SUSP_AGE_GROUP'.
  Examples being removed: ['-969', '2020', '-977', '-962', '2019', '-71', '-12', '-942', '1020', '-965']...
Removed 215 rows with invalid age formats in 'SUSP_AGE_GROUP'.
Found 131 invalid age formats in 'VIC_AGE_GROUP'.
  Examples being removed: ['943', '-967', '-4', '-958', '-968', '949', '-948', '-973', '-2', '932']...
Removed 131 rows with invalid age formats in 'VIC_AGE_GROUP'.
Total rows removed due to invalid age formats: 346
Rows remaining: 2506411


## 2.8 Clean: Remove Duplicate Records

Remove exact duplicate rows, keeping only the first occurrence of each unique row.

In [23]:
print("\n2.8 REMOVING EXACT DUPLICATE RECORDS")

if original_row_count > 0: # Proceed only if df was loaded
    rows_before_duplicates = len(df)
    # Calculate the number of duplicate rows (excluding the first instance)
    initial_duplicate_count = df.duplicated().sum()
    print(f"Found {initial_duplicate_count} exact duplicate rows (excluding first occurrences)." )
    removal_key = 'Exact Duplicates'
    cleaning_rows_removed[removal_key] = 0 # Initialize

    if initial_duplicate_count > 0:
        # Remove exact duplicates, keeping the first instance
        df = df.drop_duplicates(keep='first')
        rows_after_duplicates = len(df)
        rows_removed = rows_before_duplicates - rows_after_duplicates

        # Store the count of removed rows
        cleaning_rows_removed[removal_key] = rows_removed
        print(f"Removed {rows_removed} exact duplicate rows.")
    else:
        print("No exact duplicate rows found to remove.")

    print(f"Rows remaining: {len(df)}")
else:
    print("Skipping step 2.8 due to DataFrame loading issues.")


2.8 REMOVING EXACT DUPLICATE RECORDS
Found 9652 exact duplicate rows (excluding first occurrences).
Removed 9652 exact duplicate rows.
Rows remaining: 2496759


## 2.9 Clean: Standardize Categorical Values (Case and Mapping)

Standardize values in categorical columns for consistency, including case conversion and value mapping.

In [24]:
print("\n2.9 STANDARDIZING CATEGORICAL VALUES (CASE AND MAPPING)")

if original_row_count > 0: # Proceed only if df was loaded
    # 1. Apply specific value mappings (case-insensitive matching, then standardize)
    standardization_map = {
        'BORO_NM': {'MN':'MANHATTAN', 'M':'MANHATTAN', 'BX':'BRONX', 'BK':'BROOKLYN', 'K':'BROOKLYN', 'QN':'QUEENS', 'Q':'QUEENS', 'SI':'STATEN ISLAND', 'R':'STATEN ISLAND'},
        'LOC_OF_OCCUR_DESC': {'FRONT OF':'FRONT', 'OPPOSITE OF':'OPPOSITE', 'REAR OF':'REAR'}
        # Add other mappings if needed based on analysis
    }

    print("Applying specific value mappings...")
    for col, mapping in standardization_map.items():
        if col in df.columns and pd.api.types.is_object_dtype(df[col]):
            # Create a mapping with uppercase keys for case-insensitive lookup
            mapping_upper_keys = {k.upper(): v for k, v in mapping.items()}
            # Get the series to work with
            col_series = df[col].astype(str)
            col_series_upper = col_series.str.upper()
            # Create a mask for rows that need mapping
            mask_needs_mapping = col_series_upper.isin(mapping_upper_keys.keys())
            count_to_map = mask_needs_mapping.sum()

            if count_to_map > 0:
                print(f"  Mapping {count_to_map} values in '{col}'...")
                # Apply mapping using the uppercase keys
                # The .map() function can apply the mapping efficiently
                # We replace only the values that need mapping to preserve original case if desired, though we convert to upper later
                df[col] = col_series.mask(mask_needs_mapping, col_series_upper.map(mapping_upper_keys))
                # Verify - df.loc[mask_needs_mapping, col] = df.loc[mask_needs_mapping, col].str.upper().map(mapping_upper_keys)
            # else: print(f"  No values requiring mapping found in '{col}'.")

        elif col in df.columns:
             print(f"  Skipping mapping for non-object column '{col}'.")
        # else: Column not found - skipped

    # 2. Convert specific columns to uppercase for final uniformity
    # Ensure all relevant categorical columns are included
    case_standardization_cols = [
        'BORO_NM', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM',
        'LAW_CAT_CD', 'OFNS_DESC', 'SUSP_RACE', 'SUSP_SEX', 'VIC_RACE', 'VIC_SEX',
        'SUSP_AGE_GROUP', 'VIC_AGE_GROUP' # Also standardize age group strings
        ]

    print("\nApplying uppercase standardization...")
    for col in case_standardization_cols:
        if col in df.columns:
            # Check if it's an object dtype before attempting string operations
            if pd.api.types.is_object_dtype(df[col]):
                 try:
                     df[col] = df[col].astype(str).str.upper()
                     # print(f"  Converted column '{col}' to uppercase.") # Optional: reduce verbosity
                 except Exception as e:
                     print(f"  Warning: Could not convert column {col} to uppercase: {e}")
            # else: Skipped non-object columns
        # else: Column not found - skipped
    print("Uppercase standardization applied.")

    print("Categorical value standardization complete.")
    print(f"Rows remaining: {len(df)}") # Row count should not change here
else:
    print("Skipping step 2.9 due to DataFrame loading issues.")


2.9 STANDARDIZING CATEGORICAL VALUES (CASE AND MAPPING)
Applying specific value mappings...
  Mapping 723310 values in 'LOC_OF_OCCUR_DESC'...

Applying uppercase standardization...
Uppercase standardization applied.
Categorical value standardization complete.
Rows remaining: 2496759


## 2.10 Final Cleaning Summary

Summarize the changes made to the dataset after all cleaning steps.

In [25]:
print("\n2.10 FINAL CLEANING SUMMARY")

if original_row_count > 0: # Proceed only if df was loaded and processed
    final_row_count = len(df)
    final_col_count = df.shape[1]
    total_rows_removed = original_row_count - final_row_count
    percent_removed = (total_rows_removed / original_row_count * 100) if original_row_count > 0 else 0

    print(f"Original dataset size: {original_row_count} rows") # Initial columns might differ if some were dropped
    print(f"Final dataset size after cleaning: {final_row_count} rows, {final_col_count} columns")
    print(f"Total rows removed: {total_rows_removed} ({percent_removed:.2f}% of original data)")

    if cleaning_rows_removed:
        print("\n--- Breakdown of Rows Removed by Cleaning Step (approximate) ---")
        # Sort by number of rows removed for clarity
        # Filter out steps that removed 0 rows
        sorted_removals = sorted(
            [(k, v) for k, v in cleaning_rows_removed.items() if v > 0],
            key=lambda item: item[1],
            reverse=True
        )
        if sorted_removals:
            for category, count in sorted_removals:
                category_percent = (count / original_row_count * 100) if original_row_count > 0 else 0
                print(f"  - {category}: {count} rows ({category_percent:.2f}% of original)")
        else:
             print("  No rows were recorded as removed by specific cleaning steps.")

        # Verify total removed matches sum of steps (approximate check, order matters)
        sum_removed_steps = sum(cleaning_rows_removed.values())
        print(f"\nSum of rows removed across tracked steps: {sum_removed_steps}")
        if total_rows_removed != sum_removed_steps:
            print(f"  Note: The total removed ({total_rows_removed}) might differ from the sum of tracked steps ({sum_removed_steps}) due to the order of operations or untracked removals. The final count ({final_row_count}) is accurate.")

    else:
        print("\nNo row removal tracking data available.")

    final_missing_values = df.isna().sum().sum()
    print(f"\nTotal missing values (NaN) remaining in cleaned dataset: {final_missing_values}")
    if final_missing_values > 0:
        print("Remaining missing values breakdown:")
        print(df.isna().sum()[df.isna().sum() > 0])
else:
    print("Skipping final summary due to DataFrame loading or processing issues.")


2.10 FINAL CLEANING SUMMARY
Original dataset size: 2512541 rows
Final dataset size after cleaning: 2496759 rows, 18 columns
Total rows removed: 15782 (0.63% of original data)

--- Breakdown of Rows Removed by Cleaning Step (approximate) ---
  - Exact Duplicates: 9652 rows (0.38% of original)
  - Placeholder (NULL) in BORO_NM: 3689 rows (0.15% of original)
  - Missing PD_CD: 2066 rows (0.08% of original)
  - Invalid Format SUSP_AGE_GROUP: 215 rows (0.01% of original)
  - Invalid Format VIC_AGE_GROUP: 131 rows (0.01% of original)
  - Missing Geographic Coordinates: 28 rows (0.00% of original)
  - Placeholder (NULL) in VIC_SEX: 1 rows (0.00% of original)

Sum of rows removed across tracked steps: 15782

Total missing values (NaN) remaining in cleaned dataset: 0


## 2.11 Scan of the Cleaned Dataset

Perform a final scan of the dataset to check the status of each column after cleaning.

In [26]:
print("\n2.11 SCAN OF CLEANED DATASET")

if original_row_count > 0 and 'df' in locals() and len(df) > 0: # Proceed only if df exists and has rows
    print("Analyzing the values in each column after cleaning...")

    # Function to limit long string values for display purposes
    def format_value_for_display(val, max_len=50):
        if pd.isna(val):
            return "<NaN>"
        val_str = str(val)
        if len(val_str) > max_len:
            return val_str[:max_len - 3] + "..."
        return val_str

    # Function to analyze and display column value information
    def analyze_column_values_post_cleaning(df, column_name):
        print(f"\n=== Column: {column_name} ===")
        col_data = df[column_name]
        num_rows = len(col_data)

        # Get data type and count of unique values
        dtype = col_data.dtype
        unique_count = col_data.nunique()
        nan_count = col_data.isna().sum()
        nan_percent = (nan_count / num_rows) * 100 if num_rows > 0 else 0

        print(f"Data type: {dtype}")
        print(f"Unique values (non-NaN): {unique_count}")
        if nan_count > 0:
            print(f"Missing values (NaN): {nan_count} ({nan_percent:.2f}%)")
        else:
            print("Missing values (NaN): 0")

        # Get value counts (including NaN if present)
        try:
             value_counts = col_data.value_counts(dropna=False)
             # Display top values
             print(f"\nTop values (showing max 15):")
             num_to_show = min(15, len(value_counts))

             for i, (value, count) in enumerate(value_counts.iloc[:num_to_show].items()):
                 percent = (count / num_rows) * 100 if num_rows > 0 else 0
                 formatted_value = format_value_for_display(value)
                 print(f"  {i+1}. {formatted_value}: {count} occurrences ({percent:.2f}%)")

             if len(value_counts) > num_to_show:
                 remaining = len(value_counts) - num_to_show
                 print(f"  ... and {remaining} more unique values")

        except Exception as e:
             # Handle potential errors with unhashable types if data is very messy
             print(f"\nCould not display value counts for {column_name}: {e}")

        # For numeric columns, show basic statistics
        if pd.api.types.is_numeric_dtype(col_data) and not col_data.isna().all():
            print("\nNumeric statistics:")
            try:
                stats = col_data.describe()
                print(f"  Count: {int(stats['count'])}")
                print(f"  Mean: {stats['mean']:.4f}")
                print(f"  Std Dev: {stats['std']:.4f}")
                print(f"  Min: {stats['min']}")
                print(f"  25%: {stats['25%']}")
                print(f"  Median (50%): {stats['50%']}")
                print(f"  75%: {stats['75%']}")
                print(f"  Max: {stats['max']}")
            except Exception as e:
                print(f"  Could not calculate numeric statistics for this column: {e}")
        elif pd.api.types.is_numeric_dtype(col_data):
             print("\nNumeric statistics: Column is numeric but contains only NaN values.")

    # Analyze each column in the cleaned dataframe
    print("\nScanning all columns in the cleaned dataset...")
    for column in df.columns:
        analyze_column_values_post_cleaning(df, column)
    print("\nClean dataset scan completed.")
elif 'df' in locals() and len(df) == 0:
    print("Skipping scan as the cleaned dataset has no rows.")
else:
    print("Skipping scan due to DataFrame loading or processing issues.")



2.11 SCAN OF CLEANED DATASET
Analyzing the values in each column after cleaning...

Scanning all columns in the cleaned dataset...

=== Column: BORO_NM ===
Data type: object
Unique values (non-NaN): 5
Missing values (NaN): 0

Top values (showing max 15):
  1. BROOKLYN: 699518 occurrences (28.02%)
  2. MANHATTAN: 606565 occurrences (24.29%)
  3. QUEENS: 546234 occurrences (21.88%)
  4. BRONX: 537491 occurrences (21.53%)
  5. STATEN ISLAND: 106951 occurrences (4.28%)

=== Column: CMPLNT_FR_DT ===
Data type: object
Unique values (non-NaN): 1827
Missing values (NaN): 0

Top values (showing max 15):
  1. 2020-06-01: 1916 occurrences (0.08%)
  2. 2020-01-01: 1889 occurrences (0.08%)
  3. 2024-03-01: 1872 occurrences (0.07%)
  4. 2023-09-01: 1842 occurrences (0.07%)
  5. 2024-07-05: 1840 occurrences (0.07%)
  6. 2024-08-01: 1831 occurrences (0.07%)
  7. 2023-06-01: 1829 occurrences (0.07%)
  8. 2024-09-20: 1827 occurrences (0.07%)
  9. 2024-11-01: 1819 occurrences (0.07%)
  10. 2023-11-01: 1

## Save cleaned data

Persist the cleaned dataset as cleaned_crime_data_processed.csv for the Data Integration notebook.

In [27]:
print("\n2.12 SAVING CLEANED DATASET")

if original_row_count > 0 and 'df' in locals() and len(df) > 0: # Proceed only if df exists and has rows
    # Define the output file path
    cleaned_output_file_name = "cleaned_crime_data_processed.csv"
    # Ensure processed_dir is defined from the Setup section
    if 'processed_dir' in locals() and os.path.isdir(processed_dir):
        cleaned_file_path = os.path.join(processed_dir, cleaned_output_file_name)
        try:
            # Save the cleaned dataframe to CSV
            df.to_csv(cleaned_file_path, index=False)
            print(f"Cleaned dataset successfully saved to: {cleaned_file_path}")
            print(f"Final dimensions: {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"Error saving cleaned dataset: {e}")
    else:
        print(f"Error: Output directory '{processed_dir}' not found or not defined. Cannot save file.")
elif 'df' in locals() and len(df) == 0:
    print("Skipping save as the cleaned dataset has no rows.")
else:
    print("Skipping save due to DataFrame loading or processing issues.")

print("\n=== CLEANING SECTION COMPLETED ===")


2.12 SAVING CLEANED DATASET
Cleaned dataset successfully saved to: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Processed\cleaned_crime_data_processed.csv
Final dimensions: 2496759 rows, 18 columns

=== CLEANING SECTION COMPLETED ===
